In [12]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
df = pd.read_csv("/Users/Abhirup/Desktop/Projects/HSBC DS Assignment/PS_20174392719_1491204439457_log.csv")
df.sort_values(by=['step'])
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})

In [3]:
df_cot=df[df['type'].isin(['CASH_OUT','TRANSFER'])]
df_cot=df_cot.reset_index(drop=True)
df_cot.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,0.0,0.00,1,0
1,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0
2,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0
3,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0
4,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0


In [4]:
print(df_cot.shape)
print(len(df_cot['step'].drop_duplicates()))

(2770409, 11)
743


In [5]:
df_cot['step'].value_counts(bins=4).sort_index()

(0.257, 186.5]     990974
(186.5, 372.0]    1317246
(372.0, 557.5]     380545
(557.5, 743.0]      81644
Name: step, dtype: int64

In [6]:
#Since Steps are hours, make bins of observations to weeks
for i in range(4):
    df_cot.loc[(df_cot['step']>186*i) & (df_cot['step']<=186*(i+1)),'week'] =i+1
df_cot['week'].value_counts().sort_index()

1.0     990974
2.0    1317246
3.0     380559
4.0      81630
Name: week, dtype: int64

In [9]:
len(df_cot)

2770409

In [77]:
for i in range(1,4):
    print("\nCustomers Transacting both in Week %s and Week %s"%(i,i+1))
    print(pd.merge(df_cot.loc[df_cot['week']==i,'nameOrig'],df_cot.loc[df_cot['week']==i+1,'nameOrig'],on='nameOrig')
      ['nameOrig'].nunique())
    print("\nCustomers Transacting both in Week %s and Week %s"%(i,i+2))
    print(pd.merge(df_cot.loc[df_cot['week']==i,'nameOrig'],df_cot.loc[df_cot['week']==i+2,'nameOrig'],on='nameOrig')
      ['nameOrig'].nunique())


Customers Transacting both in Week 1 and Week 2
628

Customers Transacting both in Week 1 and Week 3
178

Customers Transacting both in Week 2 and Week 3
210

Customers Transacting both in Week 2 and Week 4
48

Customers Transacting both in Week 3 and Week 4
10

Customers Transacting both in Week 3 and Week 5
0


In [ ]:
df_cot_f=pd.DataFrame()
for i in range(990974,len(df_cot)):
    
    clear_output(wait=True)
    
    name=df_cot.iloc[i,3]
    step=df_cot.iloc[i,0]
    df_t1=df_cot.loc[(df_cot['step']>=step-186)&(df_cot['step']<step)&(df_cot['nameOrig']==name)
                    ,['step','nameOrig','amount']].groupby('nameOrig',as_index=False)['amount'].sum()
    df_t2=df_cot.loc[(df_cot['step']>=step-186)&(df_cot['step']<step)&(df_cot['nameOrig']==name)
                     ,['step','nameOrig','amount']].groupby('nameOrig',as_index=False)['amount'].count()
    
    df_t3=df_cot.iloc[[i]]
    
    if df_t1.empty==True:
        df_t3['$TransL1W']=0
        df_t3['NumTransL1W']=0
        df_cot_f=df_cot_f.append(df_t3)
    elif df_t1.empty==False:
        df_t1=df_t1.rename(columns={'amount':'$TransL1W'})
        df_t2=df_t2.rename(columns={'amount':'NumTransL1W'})
        df_t=pd.merge(df_t1,df_t2,on='nameOrig')
        df_t3=pd.merge(df_t3,df_t,how='left',on='nameOrig')
        df_cot_f=df_cot_f.append(df_t3)
        
    print("Current Progress:",np.round((i-990973)/(len(df_cot)-990973)*100,2),'%')

Current Progress: 7.76 %


In [ ]:
df_cot_f.to_csv('/Users/Abhirup/Desktop/Projects/HSBC DS Assignment/df_cot_f.csv', index=False)